//+------------------------------------------------------------------+
//|                                             TickPOC_Line_EA.mq5  |
//|                        Pure MQL5, MT5 only                       |
//+------------------------------------------------------------------+
#property strict
#property version   "1.00"
#property description "EA that calculates tick-volume POC and plots as one line"

//--- input parameters
input ENUM_TIMEFRAMES TimeFrame = PERIOD_M1;  // Timeframe for analysis
input int               BarsToAnalyze = 500; // Number of bars to calculate POC
input color             LineColor = clrRed;  // POC line color
input int               LineWidth = 2;       // POC line width

//--- global variables
double PricePOC[];      // POC price per bar
string PocLineName = "POC_Line";

//+------------------------------------------------------------------+
//| Expert initialization function                                   |
//+------------------------------------------------------------------+
int OnInit()
  {
   // Create a polyline object for POC
   if(!ObjectCreate(0,PocLineName,OBJ_POLYLINE,0,0,0))
      Print("Failed to create POC polyline.");
   ObjectSetInteger(0,PocLineName,OBJPROP_COLOR,LineColor);
   ObjectSetInteger(0,PocLineName,OBJPROP_WIDTH,LineWidth);
   ObjectSetInteger(0,PocLineName,OBJPROP_STYLE,STYLE_SOLID);
   
   EventSetTimer(1);  // update every second
   return(INIT_SUCCEEDED);
  }
//+------------------------------------------------------------------+
//| Expert deinitialization function                                 |
//+------------------------------------------------------------------+
void OnDeinit(const int reason)
  {
   EventKillTimer();
   ObjectDelete(PocLineName);
  }
//+------------------------------------------------------------------+
//| Timer function                                                   |
//+------------------------------------------------------------------+
void OnTimer()
  {
   CalculatePOC();
  }
//+------------------------------------------------------------------+
//| Calculate POC function                                           |
//+------------------------------------------------------------------+
void CalculatePOC()
  {
   MqlRates rates[];
   int copied = CopyRates(_Symbol, TimeFrame, 0, BarsToAnalyze, rates);
   if(copied <= 0) return;

   ArrayResize(PricePOC,copied);

   for(int i=0; i<copied; i++)
     {
      double maxVolume = 0;
      double pocPrice = rates[i].close; // fallback
      
      double step = SymbolInfoDouble(_Symbol,SYMBOL_POINT)*10;
      double low = rates[i].low;
      double high = rates[i].high;
      
      int bins = int((high-low)/step)+1;
      double volumes[];
      ArrayResize(volumes,bins);
      
      // Assign all tick volume to middle bin (approximation)
